In [1]:
import torch
import time
from torch import nn

# === импорт твоего слоя INT8 ===
from conv_gemm.baseline_layers.triton_conv2d import TritonConv2d

In [2]:
torch.manual_seed(0)
device = "cuda"

In [3]:
N = 1
Cin = 3
Cout = 8
H = W = 32
K = 3


In [4]:
x_fp16 = torch.randn(N, Cin, H, W, device=device, dtype=torch.float16)
def bench_ms(fn, iters=50):
    torch.cuda.synchronize()
    t0 = time.time()
    for _ in range(iters):
        fn()
    torch.cuda.synchronize()
    return (time.time() - t0) * 1000.0 / iters